In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    print(e)

2023-03-16 07:08:54.530896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 07:08:54.652428: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-16 07:08:55.290729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bagnol/miniconda3/envs/anagoup/lib/
2023-03-16 07:08:55.290792: W tensorflow/com

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [3]:
from gensim.models import FastText

model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [4]:
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=20)

(3048969, 6991040)

In [11]:
import importlib
from modelli import wrapper, utils
importlib.reload(wrapper)
from modelli.wrapper import Sequence


lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)

In [12]:
import tensorflow as tf
lstm.model.call((tf.keras.Input(shape=[None]),tf.keras.Input(shape=[None,None])),training=False)
# 47 = lenght of longer sequence
# 23 = lenght of longher word

<KerasTensor: shape=(None, None, 49) dtype=float32 (created by layer 'crf_3')>

In [ ]:
import importlib
import modelli.trainer
importlib.reload(modelli.trainer)
from modelli.trainer import Trainer



trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=10, batch_size=128,verbose=1,shuffle=True)


In [ ]:
from keras.backend import batch_get_value
import pickle

lstm.save("param","weights.h5","preprocessor")
symbolic_weights = getattr(lstm.model.optimizer, 'weights')
weight_values = batch_get_value(symbolic_weights)
with open('optimizer.pkl', 'wb') as f:
    pickle.dump(weight_values, f)
